### Imports

In [ ]:
import cv2
import torch
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
from PIL.ImageOps import invert

### Image preprocessing

In [ ]:
test_image = 'test_image.jpg'

# convert to grayscale
img = Image.open(test_image).convert('L')
img = invert(img)
img = ImageEnhance.Contrast(img).enhance(2)
img = img.point(lambda p: p > 220 and 255)
img = img.filter(ImageFilter.SMOOTH)

# show image
img.show()

### Inference

In [ ]:
# Get the model
model_path = 'best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)
model.eval()

In [ ]:
# Inference
results = model(img)

# Print and show results
print(results.pandas().xyxy)
results.show()

### Remove components

In [ ]:
# Remove pixels in bounding boxing from image
def remove_pixels(image, results, padding=10):
    # Get bounding boxes
    boxes = results.xyxy[0]

    # Convert image to numpy array
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Add padding to image on all sides
    image = cv2.copyMakeBorder(image, padding, padding, padding, padding, cv2.BORDER_CONSTANT, value=[0, 0, 0])

    # Remove pixels in bounding boxes
    for box in boxes:
        # Convert coordinates to integers
        x1, y1, x2, y2 = map(int, box[0:4])  
        # Add padding to the bounding boxes
        image[y1-padding:y2+padding, x1-padding:x2+padding] = 0

    # Remove padding
    image = image[padding:-padding, padding:-padding]

    # Convert image back to PIL image
    image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    return image

# Remove pixels in bounding boxes
image = remove_pixels(img, results)

# # Show image
# image.show()

### Line detection

In [ ]:
# convert image to numpy array
image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

# convert image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# blur image
gray = cv2.GaussianBlur(gray, (5, 5), 0)

# find edges in image
edges = cv2.Canny(gray, 50, 150)

# find lines in image
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=15, minLineLength=30, maxLineGap=10)

# draw lines on image
for points in lines:
    x1, y1, x2, y2 = points[0]
    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# convert image back to PIL image
image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

# show image
image.show()

In [ ]:
# Loop through all the lines and find all intersections
intersections = []

for i in range(len(lines)):
    for j in range(i+1, len(lines)):
        x1, y1, x2, y2 = lines[i][0]
        x3, y3, x4, y4 = lines[j][0]

        # Find intersection
        d = (x1-x2)*(y3-y4) - (y1-y2)*(x3-x4)
        if d:
            x = ((x1*y2-y1*x2)*(x3-x4)-(x1-x2)*(x3*y4-y3*x4))/d
            y = ((x1*y2-y1*x2)*(y3-y4)-(y1-y2)*(x3*y4-y3*x4))/d
            intersections.append((int(x), int(y)))
    
# Draw intersections on image
for point in intersections:
    cv2.circle(image, point, 5, (0, 0, 255), -1)

# convert image back to PIL image
image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

# show image
image.show()